# Simulate Real-world Deployment

In part-1, we learned about the simulator, where we simulated federated learning training without worrying about how the federated learning system is set up and deployed.

In a real-world deployment for cross-silo deployment, several institutions (hospitals or banks) participate in the federated system. What we go through is the setup/deployment process:

* Provision: create a set of packaged software, also known as the NVIDIA FLARE startup-kit
* Distribution: transfer the software packages (startup-kit) to each site
* Start: start the software using `<startup-kit>/start/start.sh` and wait for job submission

Before connecting multiple machines, we can simulate this setup and deployment on a local computer. In NVFLARE, this is called Proof of Concept (POC) mode.

In this section, we will explore the POC-based FL system.


## Proof of Concept (POC) Command

[POC command](https://nvflare.readthedocs.io/en/main/user_guide/nvflare_cli/poc_command.html) provides a set of command to create different software packages and simulate client and server.  You can also find the tutorials in [here](../../../../setup_poc.ipynb) on how to setup POC. 

> With POC mode, it is ideally to use terminal instead of notebook for setup, as there are some inconsistent behaviors when running scripts.

### POC Prepare

#### POC command first look

First let's prepare for the POC setup. We like to use the same example we had before with five clients: which means we should have five sites, where each site has its own startup kit package.

To simulate such setup, we use the following command


```
nvflare poc prepare -n 5

```
let run this from a separate terminal. here is the output 

```


In [ ]:
! nvflare poc prepare -n 5


Take a look at the generated folder structure: 


```
$ tree /tmp/nvflare/poc/example_project/prod_00

/tmp/nvflare/poc/example_project/prod_00
├── admin@nvidia.com
│   ├── local
│   ├── startup
│   └── ...
| 
├── server
│   ├── local
│   ├── readme.txt
│   ├── startup
│   │   ├── ....
│   └── ...

├── site-1
│   ├── local
│   ├── readme.txt
│   ├── startup
│   │   └── ...
│   └── ...
├── site-2
│   ├── local
│   ├── readme.txt
│   ├── startup
│   │   ├── ...
│   └── ...
├── site-3
│   ├── local
│   ├── readme.txt
│   ├── startup
│   │   └── ...
│   └── ...
├── site-4
│   ├── local
│   ├── readme.txt
│   ├── startup
│   │   └── ...
│   └── transfer
└── site-5
    ├── local
    ├── readme.txt
    ├── startup
    │   └── ...
    └── ...

```


Notice the command creates a working directory at ```/tmp/nvflare/poc/example_project/prod_00```


based on the default values:

* POC workspace = "/tmp/nvflare/poc"
* project_name = "example_project"
* and default folder "prod_00"


All sites are with default names starting with "site-".

The actual process of generating such software packages (startup kit) is called **"provision"**. The POC is a special mode of provision, where both client and server are at localhost.

Each site (site-N and server) represents one location in the federated learning system and runs a federated learning client.


#### Simulating the real-world deployment







In NVFLARE, we define a project.yml to describe the type of setup we would like to have in the federated deployment, here is an example: 


In [ ]:
! cat code/project.yml

notice in this project file, we define **three** sites:

**nonprofit-server** 

    served as Federated learning Server, it is belong to nonprofit organizaton
    
**us_hospital** 

    served as client, belong to us-hospital 

**europe-hospital**

    served as client, belong to eu-hospital 
    
We also define few users:

* nonprofit.org team

  **admin@nonprofit.org** -- project admin

* us_hospital team

    **admin@hospital.org.us** -- site admin  ( aka Org Admin)

    **lead@hospital.org.us** -- lead member

    **member@hospital.org.us**  -- member 
    
* eu_hospital team

    **admin@hospital.org.eu**  -- site admin 

    **lead@hospital.org.eu**   -- lead member

    **member@hospital.org.eu** -- member 


Now let prepare the startup kit with POC command, see what we have



In [ ]:
    
! echo 'y' | nvflare poc prepare -i code/project.yml



In [ ]:
! tree /tmp/nvflare/poc/health_project/prod_00

Now you see site names are now match the ones we defined in the project.yml file instead of "server",. "site-1", ... etc.

#### Prepare with Named Clients

If you just want to have a default deployment but specify the client site names (instead of using default site-1,2 etc.) without writing a project.yaml file, you can do the following:


nvflare poc prepare -c [CLIENTS ...]

In [ ]:
! echo 'y' | nvflare poc prepare -c hospital-1 hospital-2
! tree /tmp/nvflare/poc/example_project/prod_00

#### POC with docker image

The -d option instructs the provision tool to create a docker.sh use the specified docker_image. The docker.sh will start in detached mode.

nvflare poc prepare -d [DOCKER_IMAGE]
This will create docker.sh for each client and server which will run docker pull docker_image from docker hub, then running the docker in detached mode. The docker_image must use nvflare. For example, we use the following docker image "nvflare/nvflare" which has nvflare preinstalled.

>Note: you should build your own image if you want to include your dependencies such as pytorch etc.



In [ ]:
! echo 'y'| nvflare poc prepare -d 'nvflare/nvflare'
! tree /tmp/nvflare/poc/example_project/prod_00


Notice the new "docker.sh", now each client will using docker run with we specified "nvflare/nvflare" docker image

If you like to see how to specify Homomorphic encryptions you can check out our [POC tutorials](https://github.com/NVIDIA/NVFlare/blob/main/examples/tutorials/setup_poc.ipynb)

### Start NVFlare system in POC mode

When starting the POC deployment, it's necessary to use a separate terminal since the nvflare poc start command will run in the foreground emitting output from the server and any connected clients.

Also note that nvflare poc start starts all clients and server as well as project admin console. All other participants (org admin, lead and member) will not started.

#### POC start clients and server without NVFLARE admin console

In many cases, we want to start the clients and server, but without the FLARE console for project admin, as we are going to interact with the system from a notebook or CLI. You first need to find out the user name for the project admin. By default, this is admin@nvidia.com

Create a terminal to execute the following command (in JupyterLab or host terminal):

    nvflare poc start -ex admin@nvidia.com

> Note: Using %%bash -bg to run the above command in a code cell may not always work

**Homework**: run the nvflare poc start command with or without the -ex option


#### POC start individial site Only 
 
nvflare poc start -p "site-name"

Now, let start the POC with default poc and run some jobs


In [ ]:
! echo y |  nvflare poc prepare -n 5
! tree /tmp/nvflare/poc/example_project/prod_00

Once the FLARE system is started, you can learn how to interact with it at next section: [interact with federated computing system](../03.3_interact_with_federated_computing_system/%20ways_to_interact_with_fl_system.ipynb)


After running with POC, you can shut it down. 

In [40]:
! nvflare poc stop

start shutdown NVFLARE
connect to nvflare server
checking running jobs
shutdown NVFLARE
waiting system to shutdown


After you are done with poc, you can clean it up

In [41]:
! nvflare poc clean

/tmp/nvflare/poc is removed
